### Note

Please view the [README](https://github.com/deeplearning4j/deeplearning4j/tree/master/dl4j-examples/tutorials/README.md) to learn about installing, setting up dependencies, and importing notebooks in Zeppelin

### Background

#### Cloud Cover
By definition, _cloud cover_ is the portion of the _sky_ covered by _clouds_ when viewed from a particular location (such as a weather station). It is usually measured in a unit called [__Okta__](https://en.wikipedia.org/wiki/Okta). Cloud cover is also helpful in determining sunshine duration as it is inversly related to cloud cover.

--- 

#### Goals
- Determining cloud cover in DL4J

## 1. Determining cloud cover in DL4J

We're going to translate [this code](https://github.com/bpark738/Cloud/tree/master/src/main/java/stat215) into scala and visualize it in zeppelin's notebook format. The code models cloud detection in polar regions based on __radiances__ recorded automatically by the ___MISR sensor___ aboard the NASA satellite, ___Terra___.


## 2. CNN in DL4J
- #### Data
    You can view or download the data from [here](https://github.com/bpark738/Cloud/tree/master/src/main/resources). The dataset has 9 train/test splits for our convenience.

- #### Data features
    - 3 satellite [images](https://github.com/bpark738/Cloud/tree/master/images).
    - __Expert labels__ used for model training for each point in image (see the images in the table below).
    - __NDAI, SD, CORR__, based on [subject knowledge](https://github.com/bpark738/Cloud/blob/master/yu2008.pdf).
    - __DF, CF, BF, AF, AN__ [(Radiance angles)](http://www-misr.jpl.nasa.gov/).

- #### Images

|Image 1|Image 2|Image 3|
|---|---|---|
|![Image 1](https://raw.githubusercontent.com/bpark738/Cloud/master/images/image1.png)|![Image 2](https://raw.githubusercontent.com/bpark738/Cloud/master/images/image2.png)|![Image 3](https://raw.githubusercontent.com/bpark738/Cloud/master/images/image3.png)|

|Figures|
|---|
|In the above table, the figure shows the regions highlighted though colored labels - ___ice___ as __'red'__, ___clouds___ as __'blue'__ and ___unknowns___ as __'green'__|


### Imports

In [5]:
import java.io.File
import java.net.URL

import org.apache.commons.io.FileUtils
import org.datavec.api.records.reader.impl.csv.CSVRecordReader
import org.datavec.api.split.FileSplit
import org.deeplearning4j.datasets.datavec.RecordReaderDataSetIterator
import org.deeplearning4j.eval.{Evaluation, ROC}
import org.deeplearning4j.nn.api.{Model, OptimizationAlgorithm}
import org.deeplearning4j.nn.conf.{NeuralNetConfiguration, Updater}
import org.deeplearning4j.nn.conf.layers.{DenseLayer, OutputLayer}
import org.deeplearning4j.nn.multilayer.MultiLayerNetwork
import org.deeplearning4j.nn.weights.WeightInit
import org.deeplearning4j.optimize.api.IterationListener
import org.nd4j.linalg.activations.Activation
import org.nd4j.linalg.lossfunctions.LossFunctions.LossFunction

### Creating the network

In [7]:
val crossValSet = "1" // You can use this to specify (1-9) which of the train/test split you want to use.

val seed = 123
val learningRate = 0.008
val batchSize = 32
val nEpochs = 10
val numInputs = 8
val numOutputs = 2
val numHiddenNodes = 50
val baseUrl = "https://raw.githubusercontent.com/bpark738/Cloud/master/src/main/resources"
val trainFileUrl = baseUrl + "/train/" + crossValSet + ".csv"
val testFileUrl = baseUrl + "/test/" + crossValSet + ".csv"

val trainFile: File = new File("train.csv")
val testFile: File = new File("test.csv")

FileUtils.copyURLToFile(new URL(trainFileUrl), trainFile)
FileUtils.copyURLToFile(new URL(testFileUrl), testFile)

val rrTrain = new CSVRecordReader(1)
rrTrain.initialize(new FileSplit(trainFile))
val trainIter = new RecordReaderDataSetIterator(rrTrain, batchSize, 0, 2)
val rrTest = new CSVRecordReader(1)
rrTest.initialize(new FileSplit(testFile))
val testIter = new RecordReaderDataSetIterator(rrTest, batchSize, 0, 2)
val conf = new NeuralNetConfiguration.Builder()
    .seed(seed)
    .optimizationAlgo(OptimizationAlgorithm.STOCHASTIC_GRADIENT_DESCENT)
    .learningRate(learningRate)
    .updater(Updater.ADAM)
    .list
    .layer(0, new DenseLayer.Builder().nIn(numInputs).nOut(numHiddenNodes).weightInit(WeightInit.XAVIER).activation(Activation.RELU).build)
    .layer(1, new OutputLayer.Builder(LossFunction.MCXENT).weightInit(WeightInit.XAVIER).activation(Activation.SOFTMAX).nIn(numHiddenNodes).nOut(numOutputs).build)
    .pretrain(true).backprop(true)
    .build
    
val model = new MultiLayerNetwork(conf)
model.setListeners(new IterationListener {
  override def invoke(): Unit = ???   
  override def iterationDone(model: Model, iteration: Int): Unit = {
    if(iteration % 2500 == 0) {
      println("Score at iteration " + iteration + " is " + model.score())
    }
  }   
  override def invoked(): Nothing = ???
})

### Training and Evaluation

In [9]:
(1 to nEpochs).foreach((epoch) => {
  println("Epoch number: " + epoch)
  model.fit(trainIter)
})

println("\nEvaluate model....")
println("Model evaluation stats:" + model.evaluate(testIter).stats(true))
testIter.reset()

## Visualizing the results

In [11]:
val roc = new ROC(100)
while (testIter.hasNext) {
  val batch = testIter.next
  val output = model.output(batch.getFeatures)
  roc.eval(batch.getLabels, output)
}
testIter.reset()

#### FINAL TEST Area Under the Curve (AUC)

In [13]:
println("\nFINAL TEST AUC: " + roc.calculateAUC)

## ROC plot

In [15]:
val df = sc.parallelize(roc.getRocCurve.getFpr zip roc.getRocCurve.getTpr).toDF("FPR","TPR")
df.registerTempTable("roc")

In [16]:
%%sql
select FPR, TPR from roc order by FPR

### Summary

In this tutorial, we learned about what cloud cover is and we trained a network in DL4J to determine cloud cover from a bunch of specified features. At the end, we did some visualizations from the results we obtained from the network.

### What's next?

- Check out all of our tutorials available [on Github](https://github.com/deeplearning4j/deeplearning4j/tree/master/dl4j-examples/tutorials). Notebooks are numbered for easy following.